<img style="float: right; margin: 45px 22px 22px 22px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="600" height="960" /> 
    
    
# <font color='navy'> Quantitative Finance

**Students:**
    
    Espinosa García, Lyha 
    Flores Ortiz, Moisés
    Lares Barragán, Andrés
    Rosas Medellín, Miriam Eunice
    Vázquez Vargas, Ana Cristina
    
**Date:**

    December 4th, 2021

**Professor:** 	
    
    José Mario Zárate Carbajal

    
# Option Greeks